In [1]:
# pip install gspread oauth2client pandas

In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

import os
import json

# Path where you want to save the file
file_path = 'response_data/references/raw/'
os.makedirs(os.path.dirname(file_path), exist_ok=True)

# Define the scope and authorize the client
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('', scope)  # Replace with your JSON file
client = gspread.authorize(credentials)

# Open the Google Sheet by name or URL
functions = ["communication","mobility", "self-care"]
# functions = ["communication"]

for function in functions:

    sheet_name = f"{function}_test_scheme"

    spreadsheet = client.open(sheet_name)  # Replace with your sheet name or use client.open_by_url('Sheet_URL')



    # Select the first sheet or by sheet title
    sheet = spreadsheet.sheet1  

    # Extract all data into a list of lists
    data = sheet.get_all_values()

    # Convert data to a Pandas DataFrame
    df = pd.DataFrame(data[1:], columns=data[0])  # The first row will be used as the header

    # Display the DataFrame
    # print(df)
    
    # Convert the DataFrame to a JSON string
    json_data = df.to_json(orient='records', indent=4)
    
    
    # Save the JSON string to a file
    output = f"{file_path}{function}.json"
    with open(output, 'w') as json_file:
        json_file.write(json_data)

    print(f'references of {function} has been exported')


references of communication has been exported
references of mobility has been exported
references of self-care has been exported


In [2]:
def format_ref(conversation):
    """
    format the right form of reference
    """
    

    patterns = {
        r"[cC]\s*[:;]": "C:",
        r"[pP]\s*[:;]": "P:"
    }

    # Replace each pattern in the conversation
    for pattern, replacement in patterns.items():
        conversation = re.sub(pattern, replacement, conversation)
        
    return conversation

In [3]:
# se="C; how do you feel about the lady in the shop who always smiles at you ?P; She is a lovely lady, a true gentle soul, that makes me happy,  C;what made you so happy about it? P: i just love true gentle souls"
# se = format_ref(se)
# print(se)

In [4]:
import json
import re

# fil_test_path = 'response_data/test_convers/'
# os.makedirs(os.path.dirname(file_path), exist_ok=True)

stat_dic = {}
# test_dic = {}
# fil_con_dic = {}

# functions = ["communication"]

ref_clean_path = 'response_data/references/filtered/'
os.makedirs(os.path.dirname(ref_clean_path), exist_ok=True)

for function in functions:
    stat_dic[function] = {}
    ref_raw_path = f'response_data/references/raw/{function}.json'

    # Read the JSON file
    with open(ref_raw_path, 'r') as json_file:
        data = json.load(json_file)

    # Filter the data
    # test_input = []
    fil_ref = []
    
    category = ""
    
    # test_conv=[]
    for dic in data:
        # Fill in the category
        
        if dic["Category"]:
            # print(dic["Category"])
            category = dic["Category"]
        else:
            # print(category)
            dic["Category"]= category
        # Check if "Conversation" is not empty
        conversation = dic.get("Conversation")
        
        # Check if either "FQ on function level" or "FQ on feedback" is not empty
        fq_function_level = dic.get("FQ on function level")
        fq_function_level = format_ref(fq_function_level)
        dic["FQ on function level"] = fq_function_level
        
        fq_feedback = dic.get("FQ on feedback")
        fq_feedback = format_ref(fq_feedback)
        dic["FQ on feedback"] = fq_feedback
        



        #Keep the dictionary only if conversation is not empty and at least one FQ field is not empty
        if conversation and (fq_function_level or fq_feedback):
            # test_input.append(conversation)
            fil_ref.append(dic)
            


    test_con_p = f"{ref_clean_path}{function}.json"
    with open(test_con_p, 'w') as json_file:
        json.dump(fil_ref, json_file, indent=4)
        print(f'filtered conversations and references of {function} have been saved')


# Assuming stat_dic is already initialized and function, cate are defined
    for dic in fil_ref:  # assuming data is a list of dictionaries
        if dic["Category"]:
            cate = dic["Category"]

        # Initialize the category in stat_dic if not already present
        if cate not in stat_dic[function]:
            stat_dic[function][cate] = {}

                # Initialize the keys with 0 if they don't exist
            stat_dic[function][cate].setdefault("Conversation", 0)
            stat_dic[function][cate].setdefault("FQ on function level", 0)
            stat_dic[function][cate].setdefault("FQ on feedback", 0)

            # Count the "Conversation"
        if dic.get("Conversation"):
            stat_dic[function][cate]["Conversation"] += 1

            # Count the "FQ on function level"
        if dic.get("FQ on function level"):
            stat_dic[function][cate]["FQ on function level"] += 1

            # Count the "FQ on feedback"
        if dic.get("FQ on feedback"):
            stat_dic[function][cate]["FQ on feedback"] += 1


        


filtered conversations and references of communication have been saved
filtered conversations and references of mobility have been saved
filtered conversations and references of self-care have been saved


In [5]:
print(stat_dic)

{'communication': {'Communicating with receiving spoken messages': {'Conversation': 4, 'FQ on function level': 4, 'FQ on feedback': 4}, 'Communicating with receiving nonverbal messages': {'Conversation': 3, 'FQ on function level': 3, 'FQ on feedback': 2}, 'Communicating with receiving formal sign language messages': {'Conversation': 2, 'FQ on function level': 2, 'FQ on feedback': 2}, 'Communicating with receiving written messages': {'Conversation': 3, 'FQ on function level': 3, 'FQ on feedback': 3}, 'Speaking': {'Conversation': 3, 'FQ on function level': 3, 'FQ on feedback': 3}, 'Non-speech vocal expression': {'Conversation': 2, 'FQ on function level': 2, 'FQ on feedback': 2}, 'Singing': {'Conversation': 3, 'FQ on function level': 3, 'FQ on feedback': 3}, 'Producing nonverbal messages': {'Conversation': 3, 'FQ on function level': 3, 'FQ on feedback': 3}, 'Producing messages in formal sign language': {'Conversation': 2, 'FQ on function level': 2, 'FQ on feedback': 2}, 'Writing messages'

In [8]:
for function in functions:

    total_conversations = 0
    total_fq_function_level = 0
    total_fq_feedback = 0

    # Calculate totals
    print()

    for category in stat_dic[function]:
        # print(stat_dic[function][category].keys())
        total_conversations += stat_dic[function][category]['Conversation']
        total_fq_function_level += stat_dic[function][category]['FQ on function level']
        total_fq_feedback += stat_dic[function][category]['FQ on feedback']

    # Print results
    print(f"Total Conversations of {function}: {total_conversations}")
    print(f"Total FQ on Function Level of {function}: {total_fq_function_level}")
    print(f"Total FQ on Feedback of {function}: {total_fq_feedback}")
    print()


Total Conversations of communication: 36
Total FQ on Function Level of communication: 36
Total FQ on Feedback of communication: 35


Total Conversations of mobility: 52
Total FQ on Function Level of mobility: 52
Total FQ on Feedback of mobility: 32


Total Conversations of self-care: 25
Total FQ on Function Level of self-care: 25
Total FQ on Feedback of self-care: 25



In [9]:
# save conversations for test
functions = ["communication", "self-care", "mobility"]
# functions = ["communication"]


for function in functions:
    
    test_con_p = f"response_data/references/filtered/{function}.json"
    # refe_path = f"response_data/references/filtered/{function}.json"
    with open(test_con_p, "r") as refer:
        refer_dics = json.load(refer)
        
    con_dic = {}
    for d in refer_dics:
        if not d["Category"] in con_dic:
            con_dic[d["Category"]]=[d["Conversation"]]
        else:
            con_dic[d["Category"]].append(d["Conversation"])

    with open(f"response_data/test_data/in_conver/{function}.json", 'w') as json_file:
        json.dump(con_dic, json_file, indent=4)
        print(f'Test conversations of {function} have been saved')

    # print(con_dic)

    num = 0
    for k,v in con_dic.items():
        num += len(v)
    print(f"Total Test Convers of {function}: {num}")


Test conversations of communication have been saved
Total Test Convers of communication: 36
Test conversations of self-care have been saved
Total Test Convers of self-care: 25
Test conversations of mobility have been saved
Total Test Convers of mobility: 52
